In [1]:
import h5py
from tensorflow import keras

In [2]:
# Load in data
data = h5py.File('../data/data.hdf5', 'r')
phases = data['phases']
positions = data['positions']
phases_train, phases_val = phases[:-2000, ...], phases[-2000:, ...]
positions_train, positions_val = positions[:-2000, ...], positions[-2000:, ...]

test_data = h5py.File('../data/test_data.hdf5', 'r')
phases_test = test_data['phases']
positions_test = test_data['positions']

In [3]:
model = keras.Sequential()

model.add(keras.layers.Resizing(128, 128, input_shape=(256, 256, 1)))
model.add(keras.layers.Conv2D(filters=32, kernel_size=7, activation='relu'))
model.add(keras.layers.BatchNormalization(axis=3))
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization(axis=3))
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization(axis=3))
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(4))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 128, 128, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 122, 122, 32)      1600      
                                                                 
 batch_normalization (BatchN  (None, 122, 122, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 61, 61, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 59, 59, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 59, 59, 64)       2

In [4]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=1, restore_best_weights=True)

In [5]:
model.compile(optimizer='adam',loss=keras.losses.MeanSquaredError(), metrics=['accuracy'])

history = model.fit(phases_train,
                    positions_train,
                    epochs=25,
                    batch_size=8,
                    validation_data=(phases_val, positions_val),
                    callbacks=[early_stopping]
                    )

Epoch 1/25
1000/1000 [==============================] - 25s 12ms/step - loss: 33.7296 - accuracy: 0.9103 - val_loss: 11.1971 - val_accuracy: 0.9440
Epoch 2/25
1000/1000 [==============================] - 11s 11ms/step - loss: 9.4395 - accuracy: 0.9520 - val_loss: 7.6586 - val_accuracy: 0.9495
Epoch 3/25
1000/1000 [==============================] - 11s 11ms/step - loss: 8.3468 - accuracy: 0.9530 - val_loss: 6.6085 - val_accuracy: 0.9415
Epoch 4/25
1000/1000 [==============================] - 11s 11ms/step - loss: 6.4993 - accuracy: 0.9610 - val_loss: 7.6011 - val_accuracy: 0.9575
Epoch 5/25
1000/1000 [==============================] - 10s 10ms/step - loss: 5.4221 - accuracy: 0.9631 - val_loss: 3.9942 - val_accuracy: 0.9675
Epoch 6/25
1000/1000 [==============================] - 10s 10ms/step - loss: 4.5208 - accuracy: 0.9645 - val_loss: 4.8391 - val_accuracy: 0.9605
Epoch 7/25
1000/1000 [==============================] - 10s 10ms/step - loss: 4.9058 - accuracy: 0.9645 - val_loss: 3.6416

In [6]:
model.save("vortex_detection", save_format="tf")

INFO:tensorflow:Assets written to: vortex_detection\assets


INFO:tensorflow:Assets written to: vortex_detection\assets
